In [110]:
import pandas as pd
import numpy as np

In [111]:
pwd

'/root/notebooks/Recommendation Systems/ml-latest-small'

In [112]:
movies = pd.read_csv(r"movies.csv")
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [113]:
all_ratings = pd.read_csv(r"all_ratings.csv")
all_ratings.tail()

,Unnamed: 0,userId,movieId,rating
100732,728,672,148626,4.0
100733,729,672,150548,3.0
100734,730,672,152091,1.0
100735,731,672,159093,3.0
100736,732,672,160438,3.0


In [114]:
all_ratings.count()

Unnamed: 0    100737
userId        100737
movieId       100737
rating        100737
dtype: int64

In [115]:
R_df = all_ratings.pivot(index = 'userId', columns ='movieId', values = 'rating').fillna(0)
R_df.tail()

movieId,1,2,3,4,5,6,7,8,9,10,...,161084,161155,161594,161830,161918,161944,162376,162542,162672,163949
userId,,,,,,,,,,,,,,,,,,,,,
668,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
669,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
670,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
671,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
672,3.0,2.5,0.0,0.0,0.0,4.0,0.0,0.0,0.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [116]:
ratings_matrix = R_df.values
user_ratings_mean = np.mean(ratings_matrix, axis = 1)
ratings_matrix = ratings_matrix - user_ratings_mean.reshape(-1, 1)

In [117]:
from scipy.sparse.linalg import svds
U, sigma, Vt = svds(ratings_matrix, k = 50)
sigma = np.diag(sigma)

In [118]:
all_user_predicted_ratings = np.dot(np.dot(U, sigma), Vt) + user_ratings_mean.reshape(-1, 1)

In [119]:
preds_df = pd.DataFrame(all_user_predicted_ratings, columns = R_df.columns)

In [120]:
def get_recommendations(preds_df, movies, user_id, all_ratings, num_recommendations=10): 
    #get the row of the matrix corresponding to the user
    sorted_user_predictions = preds_df.iloc[user_id - 1].sort_values(ascending=False)
    #turn the series data into a dataframe
    sorted_predicts = pd.DataFrame({'movieId':sorted_user_predictions.index, 'prediction':sorted_user_predictions.values})
    #join all predicts with all predictions for the user
    predicted = sorted_predicts.merge(movies, how = 'inner', left_on = 'movieId', right_on = 'movieId')
    #get the actual movies the user has rated
    my_ratings = all_ratings[all_ratings.userId == user_id]
    
    print('User {0} has already rated {1} movies.'.format(user_id, my_ratings.shape[0]))
    
    #remove the rated movies from the predictions and sort.
    new_predics = predicted[~predicted['movieId'].isin(my_ratings['movieId'])].sort_values('prediction',ascending=False)
    #TODO: return already rated and their predictions
    return new_predics[:num_recommendations]

In [121]:
my_predictions = get_recommendations(preds_df,movies, 672, all_ratings)
my_predictions

User 672 has already rated 733 movies.


,movieId,prediction,title,genres
102,1625,3.247079,"Game, The (1997)",Drama|Mystery|Thriller
158,4022,2.893171,Cast Away (2000),Drama
178,3408,2.742362,Erin Brockovich (2000),Drama
230,3897,2.413074,Almost Famous (2000),Drama
240,68157,2.361861,Inglourious Basterds (2009),Action|Drama|War
254,4226,2.287326,Memento (2000),Mystery|Thriller
270,5218,2.234953,Ice Age (2002),Adventure|Animation|Children|Comedy
295,8529,2.126139,"Terminal, The (2004)",Comedy|Drama|Romance
308,7143,2.085543,"Last Samurai, The (2003)",Action|Adventure|Drama|War
312,1288,2.067956,This Is Spinal Tap (1984),Comedy
